# Keyence Multipoint Stitcher

This notebook contains a script for converting loose image files produced by the Keyence BZX800 into stitched z-stack image pyrimids.

This script was written by Sam Budoff for the Poleg-Polsky lab's retina mapping project in February 2024

This script makes use of the ImageJ stitching plugin which should be cited as: Preibisch, S., Saalfeld, S., & Tomancak, P. (2009). Globally optimal stitching of tiled 3D microscopic image acquisitions. Bioinformatics, 25(11), 1463–1465. doi:10.1093/bioinformatics/btp184

Note, a limitation of this stitching plugin is it is very memory intensive, therefore it is necesary to use a PC with at least 10% more than twice the size of all image files in a multipoint subdirectory. For example, if your loose image tiles for a region of interest constitute 50 gb, you will need a computer with >110 gb RAM to run this stitching plugin. Additionally, the stitched image saving is very slow. That said, the script is set up to stitch each multipoint region of interest sequentially, so the memory considerations are related only to your largest region of interest.

## Script Flow:

1) Load libraries and functions
2) Define the root directory containing all multipoint aquistion subdirectories
3) Create a dictionary containing paths to all loose image files
4) Extract the metadata from teh respective multipoint aquisitions
5) Create Z-stacks from each channel and each position
6) Merge all channels from the z-stacks
7) Compute the position of each tiles and store this as a FIJI Stitching Macro txt list
8) Create a headless FIJI instance to run the stitcher
9) Convert the stitched z-stacks into ome.tif pyrimids compatible with Xenium software
10) Optional, delete intermediate files

### Paths and Hyperparameters:

Please adjust the path to your multipoint directory and any other hyperparameters here.

Note that the default images from Keyence have a 30% overlap and unless the color was specified as white for a channel the images are saved as RGB. I recommend saving the images as white as I have not validated the RGB to greyscale function behaves perfectly since this is an unusual choice on the developer's part for a flourescent microscope.

Raster means aquisition followed an efficient "snake" pattern.

RAM_allowance specifies what percent of system RAM ImageJ can use for stitching. I find 95% to be safe with a computer that has 126 gb total, and would caution against setting much higher as this will cause a crash when it is particularly high. For exammple, I caused a crash to occur during testing when allowing 124 gb on a large stack test. If your image regions are too large and RAM is set appropriately, ImageJ will eventually stop on its own with a warning. If this happens it will be best to check how big your merged stacks directory is and ensure it is less than half the size of the RAM you allowed. If a computer with more RAM is unavailable, you will need to subset your original directory as needed being sure to duplicate the .gci archive file with metadata into each.

If you do not need an image pyrimid or want to create one in a different software package, create_pyrimid can be set to false resulting in only the big tif output stored in the zstacks/Merged_stacks subdirectory. Note Image Pyrimid generation takes a very long time but for some applications is worth it. Note Pyrimids are generated with QuPath as this software's implementation is more than ten times faster than tifffile, even with GPU accelleration. Be sure to cite and follow the developer's instructions to install and make excecutable https://github.com/qupath/qupath/releases/tag/v0.5.0 Bankhead, P. et al. QuPath: Open source software for digital pathology image analysis. Scientific Reports (2017).https://doi.org/10.1038/s41598-017-17204-5

Optionally, you can automatically delete all intermediate files generated in this script. To be clear this includes the single channel z-stacks at each tile location, the 4D tiles at each location, the IJ stitching coordinates, and the IJ Big Tif file. Only specify this if you are either happy to rerun the computation in the future and have a finite amound of storage on your computer as the IJ stitching alone can take 10s of minutes.

In [16]:
# Set to False to be prompted in the cell at setp 2 for manual entry
directory_path = '/home/sam/scRNAseq/Xenium/IHCRegistration/Retina1/' #False 
overlap = 0.3 # Note this overlap is imperfect due to physical variation during aquisition
raster = True # This is the observed default "snake" behavior of the BZX800 
RAM_allowance = 0.95 # This specifies what percent of your system RAM ImageJ can use
qupath_executable = '/opt/QuPath/bin/QuPath' # Define the QuPath executable path
create_pyrimid = True # This will make an .ome.tif image pyrimid but will take a very long time
delete_intermediates = False # WARNING: This will leave you ONLY the .ome.tif pyrimid and original Keyence Images

### Step 1: Load Functions:

In [2]:
import os, re, gc, zipfile, tempfile, json, cv2, sys, scyjava, psutil, subprocess
import tifffile as tf
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed
import xml.etree.ElementTree as ET
import imagej as ij

In [20]:
###################################################################################################################
####################################### Misc Helper Functions #####################################################
###################################################################################################################

def extract_subdir(directory):
    "Helper function for getting subdir from full dir before saving 4D stitch"
    # Split the directory string into parts based on '/'
    parts = directory.split('/')
    # Find the index of 'zstacks' to locate the position of 'Merged_stacks' and the target subdir
    try:
        zstacks_index = parts.index('zstacks')
        # The target subdir is one directory before 'zstacks'
        subdir = parts[zstacks_index - 1]
        return subdir
    except ValueError:
        # 'zstacks' not found in the path
        return "zstacks directory not found in the path"
    
def reorganize_array_for_imagej(zstack, current_axes='CZYX', target_axes='ZCYX'):
    """Reorganize the array from the current axes order to the target axes order."""
    # Create a mapping from target axes to current indices
    axis_map = {axis: current_axes.index(axis) for axis in target_axes if axis in current_axes}
    # Generate the new order of axes as indices
    new_order = [axis_map[axis] for axis in target_axes if axis in current_axes]
    # Reorganize the array
    return np.transpose(zstack, axes=new_order)

def tif_format_saver(filename, zstack, metadata=None, axes='ZYX', is_big_tiff=False):
    '''This helper function standardizes regular tif saving, including enhanced metadata handling.'''
    if not is_big_tiff:
        estimated_size_bytes = zstack.nbytes
        is_big_tiff = estimated_size_bytes > 4 * 1024**3  # Greater than 4GB
        adjusted_axes = ''.join([a for a in 'TZCYXS' if a in axes])
    
    if adjusted_axes != axes:
        zstack = reorganize_array_for_imagej(zstack, current_axes=axes, target_axes=adjusted_axes)

    if metadata is None:
        metadata = {}

    # ImageJ-specific metadata
    imagej_metadata = {
        'axes': adjusted_axes,
        'spacing': metadata.get('spacing', 1.0),
        'unit': 'um',
        'ImageJ': '1.52p',
        'hyperstack': 'true',
        'frames': metadata.get('frames', 1),
        'mode': 'composite',
        'loop': False,
    }

    # Extract and remove keys not directly supported by ImageJ from metadata
    custom_metadata = {k: v for k, v in metadata.items() if k not in imagej_metadata}
    # Convert custom metadata to a JSON string for embedding
    custom_metadata_str = json.dumps(custom_metadata)
    
    # Save the image using imwrite with combined metadata
    tf.imwrite(
        filename + (".tif" if not is_big_tiff else ".btf"),
        zstack,
        imagej=True,
        resolution=(1./metadata.get('spacing', 1.0), 1./metadata.get('spacing', 1.0)),
        metadata={'info': custom_metadata_str, **imagej_metadata},
        photometric='minisblack',
        bigtiff=is_big_tiff
    )

    print(f"Saved {filename} with metadata as {'BigTIFF' if is_big_tiff else 'TIFF'}")

################################################################################################################
##################################  FilePath Extracting Functions ##############################################
################################################################################################################

def find_files_and_organize(directory_path=False, 
                            base_pattern=r"Image_\d{5}_Z\d{3}_CH\d\.tif$", show_files=True):
    '''This function creates a dictionary organizing all file paths of interest
        base_pattern is the defult BZX saving style with Image followed by tile position, z positon, and channel
        Note that for multiD aquisitions in between Image and tile position the multiD location is stored, 
            this is accounted for so do not modify the pattern for that'''
    if not directory_path:
        # Prompt the user for a directory path if not specified
        directory_path = input("Please enter the directory path: ")
    # Initialize the tiles dictionary
    tiles = {}
    def process_directory(path, subdir="", pattern=base_pattern):
        '''This helper function handles the multiD aquisition related subdirectory adjustments'''
        # Ensure the tiles dictionary accounts for the current processing level (subdirectory or root)
        if subdir not in tiles:
            tiles[subdir] = {}
        # Adjust the pattern for subdirectory if needed
        if subdir:
            # Modify the pattern to include the subdirectory prefix
            pattern = re.sub(r"^Image_", f"Image_{subdir}_", pattern)
        adjusted_pattern = re.compile(pattern)
        # Check if a "zstacks" subdirectory exists, and create one if it doesn't
        zstacks_path = os.path.join(path, "zstacks")
        if not os.path.exists(zstacks_path):
            os.makedirs(zstacks_path)
            print(f"'zstacks' subdirectory created at: {zstacks_path}")
        else:
            print(f"'zstacks' subdirectory already exists at: {zstacks_path}")
        # Read all "*.tif" file names in the directory or subdirectory
        tif_files = [f for f in os.listdir(path) if f.endswith('.tif')]
        # Check each file name against the adjusted pattern and organize them
        valid_files = [f for f in tif_files if adjusted_pattern.match(f)]
        for file in valid_files:
            # Extract channel and location from the file name
            if subdir=="":
                match = re.search(r"Image_(\d{5})_Z\d{3}_CH(\d)\.tif$", file)
            else:
                match = re.search(r"Image_(\w+)_(\d{5})_Z\d{3}_CH(\d)\.tif$", file)
            if match:
                if subdir=="":
                    location, channel = match.groups()
                else:
                    subdir, location, channel = match.groups()
                channel = int(channel)  # Convert channel to integer for consistent dictionary keys
                if channel not in tiles[subdir]:
                    tiles[subdir][channel] = {}
                if location not in tiles[subdir][channel]:
                    tiles[subdir][channel][location] = []
                tiles[subdir][channel][location].append(os.path.join(path, file))
    # Determine if there are any subdirectories
    subdirectories = [d for d in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, d)) and d != "zstacks"]
    if subdirectories:
        for subdir in subdirectories:
            subdir_path = os.path.join(directory_path, subdir)
            process_directory(subdir_path, subdir, base_pattern)
    else:
        # Process the root directory as if it's a "subdirectory" with an empty name
        process_directory(directory_path, "", base_pattern)
    def sort_dict(d):
        """This helper function recursively sorts dictionary by keys so the tiles/slices/channels are in order"""
        sorted_dict = dict()
        for key in sorted(d.keys()):
            if isinstance(d[key], dict):
                sorted_dict[key] = sort_dict(d[key])
            else:
                sorted_dict[key] = sorted(d[key]) if isinstance(d[key], list) else d[key]
        return sorted_dict
    tiles = sort_dict(tiles)
    # Optionally print out the organized files
    if show_files:
        for subdir in tiles:
            print(f"Subdirectory '{subdir}':")
            print(f"  Channels: {list(tiles[subdir].keys())}")
            print(f"    Locations:\n       {list(tiles[subdir][list(tiles[subdir].keys())[0]].keys())}")
    return tiles

################################################################################################################
##################################  MetaData Extracting Functions ##############################################
################################################################################################################

def extract_rows_columns_from_gci_archive(directory_path, subdir=""):
    '''This function reads the keyence .gci file for metadata related to tile mosaic'''
    # Adjust the .gci file path based on the presence of a subdirectory
    gci_file_name = 'Image.gci' if subdir == "" else f'Image_{subdir}.gci'
    gci_file_path = os.path.join(directory_path, gci_file_name) if subdir else os.path.join(directory_path, gci_file_name)
    with tempfile.TemporaryDirectory() as temp_dir:
        with zipfile.ZipFile(gci_file_path, 'r') as zip_ref:
            zip_ref.extractall(temp_dir)
        properties_xml_path = os.path.join(temp_dir, 'GroupFileProperty', 'ImageJoint', 'properties.xml')
        tree = ET.parse(properties_xml_path)
        root = tree.getroot()
        row_element = root.find('.//Row')
        column_element = root.find('.//Column')
        rows = int(row_element.text) if row_element is not None else None
        columns = int(column_element.text) if column_element is not None else None
    return (rows, columns)

def extract_channel_comment(directory_path, channel_number, subdir=""):
    '''This function reads the keyence .gci file for metadata related to channel filter cube identity'''
    gci_file_name = 'Image.gci' if subdir == "" else f'Image_{subdir}.gci'
    gci_file_path = os.path.join(directory_path, gci_file_name) if subdir else os.path.join(directory_path, gci_file_name)
    with tempfile.TemporaryDirectory() as temp_dir:
        with zipfile.ZipFile(gci_file_path, 'r') as zip_ref:
            zip_ref.extractall(temp_dir)
        properties_xml_path = os.path.join(temp_dir, 'GroupFileProperty', f'Channel{channel_number}', 'properties.xml')
        tree = ET.parse(properties_xml_path)
        root = tree.getroot()
        comment_element = root.find('.//Comment')
        comment = comment_element.text if comment_element is not None else None
    return comment

def sanitize_text(text):
    '''Remove non-ASCII characters from the text'''
    return ''.join(char for char in text if ord(char) < 128)

def extract_lens_parameters(directory_path, subdir=""):
    '''This function reads the keyence .gci file for metadata related to the objective'''
    gci_file_name = 'Image.gci' if subdir == "" else f'Image_{subdir}.gci'
    gci_file_path = os.path.join(directory_path, gci_file_name)
    with tempfile.TemporaryDirectory() as temp_dir:
        with zipfile.ZipFile(gci_file_path, 'r') as zip_ref:
            zip_ref.extractall(temp_dir)
        properties_xml_path = os.path.join(temp_dir, 'GroupFileProperty', 'Lens', 'properties.xml')
        tree = ET.parse(properties_xml_path)
        root = tree.getroot()
        lens_parameters = {}
        for param in ['LensName', 'TypeName', 'Magnification', 'NumericalAperture', 'WorkingDistance', 'LiquidImmersion', 'PhaseContrast', 'RevolverPosition']:
            element = root.find(f'.//{param}')
            # Sanitize the text to remove non-ASCII characters
            lens_parameters[param] = sanitize_text(element.text) if element is not None else None
    return lens_parameters

def extract_metadata_from_random_tif(tiles):
    """
    Extracts resolution, image dimensions, compression, samples per pixel, and other metadata
    from a random TIFF file in the tiles structure.
    
    Parameters:
    - tiles: Dictionary of organized TIFF files.
    
    Returns:
    - metadata: Dictionary containing the extracted metadata.
    """
    # Identify image tile metadata
    ch = list(tiles.keys())[0]  # Choosing the first channel for simplicity
    loc = list(tiles[ch].keys())[0]  # Choosing the first location for simplicity
    file_path = tiles[ch][loc][0]  # Selecting the first file for simplicity
    # Open the TIFF file and extract metadata
    with tf.TiffFile(file_path) as tif:
        page = tif.pages[0]  # Assuming we're only interested in the first page
        # Initialize a dictionary to store the extracted metadata
        metadata = {
            'Width': page.imagewidth,
            'Height': page.imagelength,
            'Compression': page.tags['Compression'].value if 'Compression' in page.tags else 'None',
            'SamplesPerPixel': page.samplesperpixel,
            'XResolution': page.tags['XResolution'].value if 'XResolution' in page.tags else (96, 1),
            'YResolution': page.tags['YResolution'].value if 'YResolution' in page.tags else (96, 1),
            'PlanarConfiguration': page.tags['PlanarConfiguration'].value if 'PlanarConfiguration' in page.tags else 'Not specified',
            'ResolutionUnit': page.tags['ResolutionUnit'].value if 'ResolutionUnit' in page.tags else 'Not specified',
            'Predictor': page.tags['Predictor'].value if 'Predictor' in page.tags else 'None',
        }
        # Convert resolution to DPI (dots per inch) if ResolutionUnit indicates it's in inches
        if metadata['ResolutionUnit'] == 2:  # 2 indicates inches
            dpi_x = metadata['XResolution'][0] / metadata['XResolution'][1]
            dpi_y = metadata['YResolution'][0] / metadata['YResolution'][1]
            metadata['DPI'] = (dpi_x, dpi_y)
            # Calculate voxel size in micrometers (µm) assuming resolution is in DPI (dots per inch) and 1 inch = 25.4 mm = 25400 µm
#             metadata['Voxel Size (µm)'] = (25400 / dpi_x, 25400 / dpi_y)
        # Attempt to extract magnification directly from the metadata (adjust as necessary)
        magnification_matches = re.search(r"Magnification: (\d+(\.\d+)?)", str(page.description))
        metadata['Magnification'] = float(magnification_matches.group(1)) if magnification_matches else 'Not specified'
    return metadata

def extract_metadata_for_all_subdirs(tiles, directory_path):
    '''This function ensures metadata is stored from each multiD location as settings may vary'''
    meta_dict = {}
    # Iterate through all subdirectories and the root directory in tiles
    for subdir in tiles:
        # Adjust directory path for the current subdirectory or keep it for the root
        current_directory_path = os.path.join(directory_path, subdir) if subdir else directory_path
        # Extract rows and columns from .gci file for the current subdir
        rows_columns = extract_rows_columns_from_gci_archive(current_directory_path, subdir)
        print(f"Subdirectory '{subdir}': Rows: {rows_columns[0]}, Columns: {rows_columns[1]}")
        # Extract metadata from a random tif in the current subdir
        im_meta = extract_metadata_from_random_tif(tiles[subdir])
        # Extract lens parameters for the current subdir
        lens_meta = extract_lens_parameters(current_directory_path, subdir)
        for key, val in lens_meta.items():
            im_meta[key] = val
        # Assuming channel extraction and comment need adjustment for subdirs
        for ch in tiles[subdir]:
            # Ensure each channel within a subdir has its metadata structure
            if ch not in meta_dict:
                meta_dict[ch] = {}
            if subdir not in meta_dict[ch]:
                meta_dict[ch][subdir] = im_meta
            # Extract channel comment for the current subdir
            filt = extract_channel_comment(current_directory_path, ch-1, subdir)
            meta_dict[ch][subdir]['Filter'] = filt
            print(f'Subdirectory "{subdir}", Channel {ch} = {filt}')
    return meta_dict

################################################################################################################
##################################  Z-Stack Making Functions ###################################################
################################################################################################################
def zStackMaker(file_paths, 
                subdir, channel, position,
                metadata, root, xenium=False):
    '''This function reads through the given file paths and creates a z-stack for a given position'''
    # Assume the file_paths are sorted as they should be at this point in the script
    n_z = len(file_paths)
    # Initialize an empty array for the z-stack
    z_stack = None
    for i, file_path in enumerate(file_paths):
        # Load each tif file
        image = tf.imread(file_path)
        # Check if the tif file has 3 color channels, and convert it to grey scale if it does
        # This has not been validated for scientific validity, best to save images with color set to white
        if image.ndim == 3 and image.shape[2] == 3:  # Check for color image
            # Convert to grayscale without changing bit depth
            image = np.mean(image.astype(np.float32), axis=2)  # Use float32 to avoid precision loss during mean calculation
            image = image.astype(np.uint16)  # Convert back to 16-bit
        # Save the 2D array as a 3D array or insert it into the correct position
        if z_stack is None:
            z_stack = np.zeros((n_z, *image.shape), dtype=np.uint16)
        z_stack[i, :, :] = image
    # Saving the z-stack with metadata
    if subdir == '':
        zstacks_path = f'{root}/zstacks/Image_{position}_CH{channel}_ZStack'
    else:
        zstacks_path = f'{root}/{subdir}/zstacks/Image_{subdir}_{position}_CH{channel}_ZStack'
    if xenium:
        xenium_format_saver(zstacks_path, z_stack, metadata)
    else:
        tif_format_saver(zstacks_path, z_stack, metadata)
    print(f"Z-stack saved as: {zstacks_path}.ome.tif")

def parallel_zStackMaker(tiles, meta_dict, root):
    '''This helper function enables parrallel processing of z-stackw'''
    # Define a wrapper function to include all required arguments for zStackMaker
    def task_wrapper(file_paths, subdir, channel, position, meta_dict, root):
        print(f"Processing position {position}, channel {channel}")
        zStackMaker(file_paths, subdir, channel, position, meta_dict[channel], root)
        print(f"Completed position {position}, channel {channel}")
    # Use ThreadPoolExecutor to parallelize tasks
    with ThreadPoolExecutor() as executor:
        # Dictionary to keep track of futures
        futures = []
        # Loop through each position and channel in the tiles dictionary
        for subdir in tiles:
            for channel in tiles[subdir]:
                for position in tiles[subdir][channel]:
                    # Extract file paths for the current position and channel
                    file_paths = tiles[subdir][channel][position]
                    # Submit task to the executor
                    futures.append(executor.submit(task_wrapper, file_paths, subdir, channel, position, meta_dict[subdir], root))
        # Wait for all futures to complete
        for future in as_completed(futures):
            # Result handling if needed
            result = future.result()  # This will also re-raise any exceptions from the task
            
def channel_metadata_merger(metadata):
    '''This function combines filter information from the metadata dicts'''
    meta_combined = metadata[list(metadata.keys())[0]].copy()
    del meta_combined['Filter'] 
    meta_combined[f'Channel'] = {'Name': [metadata[i]['Filter'] for i in meta_dict[subdir]]}
    for i in meta_dict[subdir]:
        meta_combined[f'Filter Cube {i}'] = metadata[i]['Filter']
    return meta_combined

def merge_zstacks_channels(channel_dict, root_directory, metadata, xenium=False):
    """
    Combines z-stacks from different channels into a 4D image for each tile and saves them with metadata.
    Parameters:
    - channel_dict: Dictionary with channels as keys and lists of z-stack file paths as values.
    - root_directory: The base directory where the 'zstacks' directory exists.
    - metadata: Dictionary containing metadata to be saved with the merged stacks.
    Output 4D Image array is CZYX, which for FIJI Hyperstack is XYZCT
    """
    # Ensure 'Merged_stacks' directory exists
    merged_stacks_dir = os.path.join(root_directory, 'Merged_stacks')
    if not os.path.exists(merged_stacks_dir):
        os.makedirs(merged_stacks_dir)
        print(f"'Merged_stacks' subdirectory created at: {merged_stacks_dir}")
    else:
        print(f"'Merged_stacks' subdirectory already exists at: {merged_stacks_dir}")
    # Process each set of z-stacks for merging
    filts = sorted(list(channel_dict.keys()))
    for i in range(len(channel_dict[filts[0]])):
        ims = [tf.imread(channel_dict[k][i]) for k in filts]
        # Load the first file to initialize the 4D array
        num_z_slices, height, width = ims[0].shape
        num_channels = len(ims)
        # Initialize the 4D array: dimensions (z, height, width, channels)
        merged_stack = np.zeros((num_channels, num_z_slices, height, width), dtype=np.uint16)
        # Load and stack each channel
        for j, im in enumerate(ims):
            # Add to the 4D array
            merged_stack[j, :, :, :] = im
        
        # Save the merged z-stack
        merged_file_path = os.path.join(merged_stacks_dir, f'Merged_Image_{i+1:05}')
        # Save stitched zstack
        if xenium:
            xenium_format_saver(merged_file_path, merged_stack, metadata)
            print(f"Merged z-stack saved as: {merged_file_path}.ome.tif")
        else:
            tif_format_saver(merged_file_path, merged_stack, metadata, axes='CZYX')
            print(f"Merged z-stack saved as: {merged_file_path}.tif")
            
################################################################################################################
##################################  ZStackStitching Functions ##################################################
################################################################################################################
def ensure_directory_exists(directory_path):
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)
    return directory_path

def organize_zstacks_by_channel(zstacks_dir):
    zstacks_files = [f for f in os.listdir(zstacks_dir) if f.endswith('.tif')]
    channel_dict = {}
    for file in zstacks_files:
        channel = int(re.search(r"_CH(\d)_", file).group(1))
        if channel not in channel_dict:
            channel_dict[channel] = []
        channel_dict[channel].append(os.path.join(zstacks_dir, file))
        channel_dict[channel]= sorted(channel_dict[channel])
#     return the dict sorted by channel id 
    return {channel: channel_dict[channel] for channel in sorted(channel_dict)}

def calculate_stitched_dimensions(shape, rows, columns, overlap):
    if len(shape) == 2:
        height, width = shape  # Assuming shape is (y, x)
    elif len(shape) == 3:
        height, width = shape[1], shape[2]  # Assuming shape is (z, y, x)
    elif len(shape) == 4:
        height, width = shape[2], shape[3]  # Assuming shape is (z, y, x)
    else:
        print('Check your image shape')
        return None
        
    stitched_height = int(height * rows - (height * overlap * (rows - 1)))
    stitched_width = int(width * columns - (width * overlap * (columns - 1)))
    return (shape[0], stitched_height, stitched_width)

def calculate_position(index, size, overlap, count):
    """Calculate the start position of each z-stack in the stitched image."""
    step = size * (1 - overlap)
    position = index * step
    return int(position)

def tile_placer(tif_files, stitched_shape, rows, columns, overlap, original_shape, output_dir,
                           raster=True):
    """This function will compute the positions for each tile in the mosaic for FIJI stitching
        Note that raster is the default behavior obsered on the Keyence but may be adjusted"""

    tile_locations = []  # List to store tile locations

    # Sort zstack_paths by Image_ID before processing
    sorted_zstack_paths = sorted(tif_files, key=lambda x: int(re.search(r"Merged_Image_(\d{5})", x).group(1)))
    for i, zstack_path in enumerate(sorted_zstack_paths):
        
        # Convert Image_ID into row and column indexes based on your specific naming convention
        image_id = int(re.search(r"Merged_Image_(\d{5})", zstack_path).group(1))
        row_index, col_index = divmod(image_id - 1, columns)  # Adjust calculation as needed

        # Apply raster ordering: reverse the order of every odd row
        if raster and row_index % 2 == 1:
            col_index = columns - 1 - col_index

        y_position = calculate_position(row_index, original_shape[2], overlap, rows)
        x_position = calculate_position(col_index, original_shape[3], overlap, columns)
        
        # Append the tile location to the list
        tile_locations.append(f"{os.path.basename(zstack_path)}; ; ({x_position}, {y_position}, 0.0)")

        # After processing all tiles, write the locations to a text file
        output_file_path = os.path.join(output_dir, "tile_locations.txt")
        with open(output_file_path, 'w') as f:
            f.write(f"# Define the number of dimensions we are working on\ndim = 3\n# Define the image coordinates (in pixels)\n")
            for location in tile_locations:
                f.write(location + "\n")

### Step 2 & 3: Specify and Load Image Paths

In [4]:
# Specify directory path to all loose images
tiles = find_files_and_organize(directory_path, show_files=True)

'zstacks' subdirectory already exists at: /home/sam/scRNAseq/Xenium/IHCRegistration/Retina1/XY03/zstacks
'zstacks' subdirectory already exists at: /home/sam/scRNAseq/Xenium/IHCRegistration/Retina1/XY04/zstacks
'zstacks' subdirectory already exists at: /home/sam/scRNAseq/Xenium/IHCRegistration/Retina1/XY02/zstacks
'zstacks' subdirectory already exists at: /home/sam/scRNAseq/Xenium/IHCRegistration/Retina1/XY01/zstacks
Subdirectory 'XY01':
  Channels: [1, 2, 3, 4]
    Locations:
       ['00001', '00002', '00003', '00004', '00005', '00006', '00007', '00008', '00009', '00010', '00011', '00012', '00013', '00014', '00015', '00016', '00017', '00018', '00019', '00020', '00021', '00022', '00023', '00024', '00025', '00026', '00027', '00028', '00029', '00030', '00031', '00032', '00033', '00034', '00035', '00036', '00037', '00038', '00039', '00040', '00041', '00042', '00043', '00044', '00045', '00046', '00047', '00048', '00049', '00050', '00051', '00052', '00053', '00054', '00055', '00056', '00057'

### Step 4: Read Metadata

In [5]:
# Initialize a meta_dict to organize metadata by subdirectory and channel
meta_dict = {}
rc_dict = {}
# Iterate through all subdirectories and the root directory in tiles
for subdir in tiles:
    current_directory_path = os.path.join(directory_path, subdir) if subdir else directory_path
    
    # Extract rows and columns for each subdirectory
    rows_columns = extract_rows_columns_from_gci_archive(current_directory_path, subdir)
    rc_dict[subdir] = rows_columns 
    print(f"Subdirectory '{subdir}': Rows: {rows_columns[0]}, Columns: {rows_columns[1]}")
    
    # Extract lens parameters for each subdirectory
    lens_meta = extract_lens_parameters(current_directory_path, subdir)
    print(f"Lens metadata for subdirectory '{subdir}':")
    for key, val in lens_meta.items():
        print(f"  {key}: {val}")
    
    # Iterate through channels within each subdirectory
    for ch in tiles[subdir]:
        # Select a random TIFF file for metadata extraction
        loc = list(tiles[subdir][ch].keys())[0]  # Select the first location for simplicity
        file_path = tiles[subdir][ch][loc][0]  # Select the first file for simplicity
        
        # Create a temporary structure for simplified access to a single channel's tiles
        temp_tiles = {ch: {loc: [file_path]}}
        im_meta = extract_metadata_from_random_tif(temp_tiles)
        
        # Merge lens metadata into image metadata
        for key, val in lens_meta.items():
            im_meta[key] = val
        
        # Extract channel comment
        filt = extract_channel_comment(current_directory_path, ch-1, subdir)
        im_meta['Filter'] = filt
        print(f"Channel {ch} metadata for subdirectory '{subdir}': Filter = {filt}")
        
        # Store the metadata in the meta_dict
        if subdir not in meta_dict:
            meta_dict[subdir] = {}
        meta_dict[subdir][ch] = im_meta

# At this point, meta_dict is structured as meta_dict[subdir][channel] = metadata,
# where metadata contains all extracted details for each channel in each subdirectory.

Subdirectory 'XY01': Rows: 7, Columns: 11
Lens metadata for subdirectory 'XY01':
  LensName: PlanApo_ 10x 0.45/4.00mm :Default
  TypeName: PlanApo
  Magnification: 1000
  NumericalAperture: 4601778099247172813
  WorkingDistance: 4616189618054758400
  LiquidImmersion: None
  PhaseContrast: None
  RevolverPosition: 1
Channel 1 metadata for subdirectory 'XY01': Filter = Cy7
Channel 2 metadata for subdirectory 'XY01': Filter = TxRed
Channel 3 metadata for subdirectory 'XY01': Filter = EYFP
Channel 4 metadata for subdirectory 'XY01': Filter = DAPI
Subdirectory 'XY02': Rows: 5, Columns: 10
Lens metadata for subdirectory 'XY02':
  LensName: PlanApo_ 10x 0.45/4.00mm :Default
  TypeName: PlanApo
  Magnification: 1000
  NumericalAperture: 4601778099247172813
  WorkingDistance: 4616189618054758400
  LiquidImmersion: None
  PhaseContrast: None
  RevolverPosition: 1
Channel 1 metadata for subdirectory 'XY02': Filter = Cy7
Channel 2 metadata for subdirectory 'XY02': Filter = TxRed
Channel 3 metadata

### Step 5: Create Z-stacks For Each Tile

In [6]:
# Create z-stacks from loose images
parallel_zStackMaker(tiles, meta_dict, directory_path)

### Step 6: Merge Z-stack Channels

In [7]:
# Store the file paths of the z-stacks
channel_dict = {}
meta_dict_4D = {}
for subdir in tiles:
    stack_path = os.path.join(directory_path, subdir, 'zstacks')
    channel_dict[subdir] = organize_zstacks_by_channel(stack_path)
    # Created merged versions of the z-stack channels for easy external viewing
    # Combine metadata
    meta_dict_4D[subdir] = channel_metadata_merger(meta_dict[subdir])
    meta_dict_4D[subdir]['slices'] = len(tiles[subdir][list(tiles[subdir].keys())[0]][list(tiles[subdir][list(tiles[subdir].keys())[0]].keys())[0]])
    merge_zstacks_channels(channel_dict[subdir], root_directory=stack_path, metadata=meta_dict_4D[subdir])
print('Z-stack Paths stored')

Z-stack Paths stored


### Step 7: Compute Tile Locations and Store as Text file

In [8]:
merged_dirs = []
for subdir in meta_dict_4D:
    
    if subdir == "":
        stack_dir = os.path.join(directory_path, 'zstacks')  # Fixed missing comma
    else:
        stack_dir = os.path.join(directory_path, subdir,'zstacks')
        
    merged_dir = os.path.join(stack_dir, 'Merged_stacks')
    merged_dirs.append(merged_dir)
    
    tif_files = sorted([f for f in os.listdir(merged_dir) if f.endswith('.tif')])

    rows, columns = rc_dict[subdir]
    zstack_shape = tf.imread(os.path.join(merged_dir, tif_files[0])).shape
    stitched_shape = calculate_stitched_dimensions(zstack_shape, rows, columns, overlap)
    print(f'Tile shapes: {zstack_shape}; Mosaic: {rows} Rows X {columns} Columns with {100*overlap}% Overlap\nStitched shape is: {stitched_shape}')
        
    # Add information to the metadata
    meta_dict_4D[subdir]['Rows'] = rows
    meta_dict_4D[subdir]['Columns'] = columns
    meta_dict_4D[subdir]['Overlap'] = overlap

    meta_dict_4D[subdir]['Original Image Shape'] = zstack_shape

    tile_placer(tif_files, stitched_shape, rows, columns, overlap, zstack_shape, merged_dir, raster=raster)

    print(f"Position {subdir} tile position file generated")
print("All tile position files computed")

Tile shapes: (12, 4, 1440, 1920); Mosaic: 7 Rows X 11 Columns with 30.0% Overlap
Stitched shape is: (12, 7488, 15360)
Position XY01 tile position file generated
Tile shapes: (12, 4, 1440, 1920); Mosaic: 5 Rows X 10 Columns with 30.0% Overlap
Stitched shape is: (12, 5472, 14016)
Position XY02 tile position file generated
Tile shapes: (12, 4, 1440, 1920); Mosaic: 9 Rows X 24 Columns with 30.0% Overlap
Stitched shape is: (12, 9504, 32832)
Position XY03 tile position file generated
Tile shapes: (12, 4, 1440, 1920); Mosaic: 8 Rows X 25 Columns with 30.0% Overlap
Stitched shape is: (12, 8496, 34176)
Position XY04 tile position file generated
All tile position files computed


### Step 8: Stitch Tiles Using Image J

In [9]:
# Calculate available memory, leaving 10% aside
mem = psutil.virtual_memory().total / (1024 ** 3)  # Convert bytes to GB
mem = str(round(RAM_allowance*mem)) # Reduce memory allowance to user specified percent of system RAM
# scyjava.config.add_options(f'-Xmx100g') # Setting 100 gb of ram for example
scyjava.config.add_options(f'-Xmx{mem}g') # Setting user specified of system RAM dynamically
ij = ij.init('sc.fiji:fiji:2.14.0') # Be sure FIJI is loaded as imageJ alone does not have the plugin
ij.getApp().getInfo(True)

'ImageJ2 2.14.0/1.54f; Java 11.0.21 [amd64]; 310MB of 115712MB'

In [10]:
plugin = "Grid/Collection stitching"
for merged_dir in merged_dirs:
    subdir = extract_subdir(merged_dir)
    # Check if the tile_locations.registered.txt file already exists
    registered_file_path = os.path.join(merged_dir, "tile_locations.registered.txt")
    if os.path.exists(registered_file_path):
        print(f"Registration already performed, skipping registration step for directory: {merged_dir}")
        args = {
        "type": "[Positions from file]",
        "order": "[Defined by TileConfiguration]",
        "directory": merged_dir,
        "layout_file": "tile_locations.registered.txt",
        "fusion_method": "[Linear Blending]",
        "compute_overlap": True,
        "subpixel_accuracy": True,
        "computation_parameters": "[Save computation time (but use more RAM)]",
        "image_output": "[Keep output virtual]",
        }
    else:
        # Perform registration
        args = {
            "type": "[Positions from file]",
            "order": "[Defined by TileConfiguration]",
            "directory": merged_dir,
            "layout_file": "tile_locations.txt",
            "fusion_method": "[Linear Blending]",
            "regression_threshold": "0.30",
            "max/avg_displacement_threshold": "2.50",
            "absolute_displacement_threshold": "3.50",
            "compute_overlap": True,
            "subpixel_accuracy": True,
            "computation_parameters": "[Save computation time (but use more RAM)]",
            "image_output": "[Keep output virtual]",
        }
    ij.py.run_plugin(plugin, args) # Stitch the image
    print(f"Image {subdir} Stitch successful")
    output = ij.py.active_imageplus() # Extract image array from IJ
    print(f"Converting Image {subdir} from JAVA to Python")
    img = ij.py.from_java(output) # Convert JAVA array to python
    output.close() # Close the ImageJ image window to free up memory
    del output # Free up memory
    img = img.values # Convert byte array to 16 bit numpy array
    img = reorganize_array_for_imagej(img, current_axes='ZYXC', target_axes='ZCYX')
    print(f"Conversion of {subdir} from JAVA to Python SUccessful, now saving")
    metadata = meta_dict_4D[subdir]
    metadata['Width'] = img.shape[3]
    metadata['Height'] = img.shape[2]
    tif_format_saver(os.path.join(merged_dir, f"Stitched_4D_ZStack_{subdir}.tif"), 
                     img, metadata=metadata, axes='ZCYX',
                     is_big_tiff=True)
    print(f"Image {subdir} 4D Stitch Saved Successfully")
    del img # Free up memory
    gc.collect()

Operating in headless mode - the original ImageJ will have limited functionality.
Operating in headless mode - the IJ class will not be fully functional.


Registration already performed, skipping registration step for directory: /home/sam/scRNAseq/Xenium/IHCRegistration/Retina1/XY01/zstacks/Merged_stacks
Stitching internal version: 1.2
Loading: /home/sam/scRNAseq/Xenium/IHCRegistration/Retina1/XY01/zstacks/Merged_stacks/Merged_Image_00001.tif ... 
Reading IFDs
Populating metadata
Checking comment style
Populating OME metadata


[java.lang.Enum.toString] Stitching internal version: 1.2[java.lang.Enum.toString] 
[java.lang.Enum.toString] Loading: /home/sam/scRNAseq/Xenium/IHCRegistration/Retina1/XY01/zstacks/Merged_stacks/Merged_Image_00001.tif ... [java.lang.Enum.toString] 
[java.lang.Enum.toString] Reading IFDs
[java.lang.Enum.toString] Populating metadata
[java.lang.Enum.toString] Checking comment style
[java.lang.Enum.toString] Populating OME metadata


1920x1440x12px, channels=4, timepoints=1 (1337 ms)
Loading: /home/sam/scRNAseq/Xenium/IHCRegistration/Retina1/XY01/zstacks/Merged_stacks/Merged_Image_00002.tif ... 
Reading IFDs
Populating metadata
Checking comment style
Populating OME metadata


[java.lang.Enum.toString] 1920x1440x12px, channels=4, timepoints=1 (1337 ms)[java.lang.Enum.toString] 
[java.lang.Enum.toString] Loading: /home/sam/scRNAseq/Xenium/IHCRegistration/Retina1/XY01/zstacks/Merged_stacks/Merged_Image_00002.tif ... [java.lang.Enum.toString] 
[java.lang.Enum.toString] Reading IFDs
[java.lang.Enum.toString] Populating metadata
[java.lang.Enum.toString] Checking comment style
[java.lang.Enum.toString] Populating OME metadata


...

[java.lang.Enum.toString] 150: 0.5020530566746075 0.7135958685532636[java.lang.Enum.toString] 
[java.lang.Enum.toString] 151: 0.5019940379798362 0.7135958685532636[java.lang.Enum.toString] 
[java.lang.Enum.toString] 152: 0.5019358730663825 0.7135958685532636[java.lang.Enum.toString] 
[java.lang.Enum.toString] 153: 0.501878248229967 0.7135958685532636[java.lang.Enum.toString] 
[java.lang.Enum.toString] 154: 0.5018212158542062 0.7135958685532636[java.lang.Enum.toString] 
[java.lang.Enum.toString] 155: 0.5017648638980402 0.7135958685532636[java.lang.Enum.toString] 
[java.lang.Enum.toString] 156: 0.5017091554527819 0.7135958685532636[java.lang.Enum.toString] 
[java.lang.Enum.toString] 157: 0.5016538787559907 0.7135958685532636[java.lang.Enum.toString] 
[java.lang.Enum.toString] 158: 0.5015993149272813 0.7135958685532636[java.lang.Enum.toString] 
[java.lang.Enum.toString] 159: 0.5015452041652176 0.7135958685532636[java.lang.Enum.toString] 
[java.lang.Enum.toString] 160: 0.5014920039666082 0


Exception: java.lang.OutOfMemoryError thrown from the UncaughtExceptionHandler in thread "Python Reference Queue"


>>>>>>>>>>>>>>>>>>>>>>>>>>>
<Out of memory>
<All available memory (115712MB) has been>
<used. To make more available, use the>
<Edit>Options>Memory & Threads command.>
>>>>>>>>>>>>>>>>>>>>>>>>>>>


[java.lang.Enum.toString] >>>>>>>>>>>>>>>>>>>>>>>>>>>[java.lang.Enum.toString] 
[java.lang.Enum.toString] <Out of memory>[java.lang.Enum.toString] 
[java.lang.Enum.toString] <All available memory (115712MB) has been>[java.lang.Enum.toString] 
[java.lang.Enum.toString] <used. To make more available, use the>[java.lang.Enum.toString] 
[java.lang.Enum.toString] <Edit>Options>Memory & Threads command.>[java.lang.Enum.toString] 
[java.lang.Enum.toString] >>>>>>>>>>>>>>>>>>>>>>>>>>>[java.lang.Enum.toString] 


java.lang.RuntimeException: java.lang.RuntimeException: Macro canceled

### Step 9: Create and Save Stitched Image Pyrimid

In [25]:
# Define the full path to your input BigTIFF file
if create_pyrimid:        
    for merged_dir in merged_dirs:
        subdir = extract_subdir(merged_dir)
        metadata = meta_dict_4D[subdir]
        input_file = os.path.join(merged_dir, f"Stitched_4D_ZStack_{subdir}.tif.btf")
        if os.path.exists(input_file):
            output_path = os.path.join(directory_path, subdir, f"Stitched_4D_ZStack_{subdir}")
            # Command and arguments for converting to OME-TIFF using QuPath
            cmd = [
                qupath_executable, 'convert-ome',
                input_file,
                output_path,
                '--pyramid-scale=2',
                '--tile-size=1024',
                '--compression=ZLIB',
                '--parallelize'
            ]
            # Execute the command
            process = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            # Check the output and error
            if process.returncode == 0:
                print("Conversion successful:", process.stdout)
            else:
                print("Error in conversion:", process.stderr)
            print(f'10X Format Image Pyrimid Generated and saved as {pyr_path}.ome.tif')
        else:
            print(f"No Big Tif found in {subdir}")

Conversion successful: 19:09:05.343 [main] [ERROR] q.l.i.writers.ome.ConvertCommand - Output file /home/sam/scRNAseq/Xenium/IHCRegistration/Retina1/XY01/Stitched_4D_ZStack_XY01.ome.tif exists!

10X Format Image Pyrimid Generated and saved as /home/sam/scRNAseq/Xenium/IHCRegistration/Retina1/XY01/Stitched_4D_ZStack_XY01.ome.tif
Conversion successful: 19:09:05.971 [main] [INFO ] qupath.lib.gui.prefs.PathPrefs - Setting default Locale to en_US
19:09:05.973 [main] [INFO ] qupath.lib.gui.prefs.PathPrefs - Setting Locale for FORMAT to en_US
19:09:05.973 [main] [INFO ] qupath.lib.gui.prefs.PathPrefs - Setting Locale for DISPLAY to en_US
19:09:05.980 [main] [INFO ] qupath.lib.common.ThreadTools - Setting parallelism to 19
19:09:05.989 [main] [INFO ] q.l.i.writers.ome.ConvertCommand - Setting tile cache size to 16088.00 MB (25.0% max memory)
19:09:06.124 [main] [INFO ] q.l.i.s.b.BioFormatsServerOptions - Setting max Bio-Formats readers to 20
19:09:06.430 [main] [INFO ] q.l.i.writers.ome.OMEPyra

### Step 10: Delete Intermediate Files

In [ ]:
if delete_intermediates:
    for subdir in tiles:
        current_directory_path = os.path.join(directory_path, subdir) if subdir else directory_path 
        # Check if the directory exists
        if os.path.exists(current_directory_path):
            # Remove the directory and all its contents
            shutil.rmtree(current_directory_path)
            print(f"Directory {current_directory_path} has been deleted.")
        else:
            print(f"Directory {current_directory_path} does not exist.")

# Experimental Code

#### If desired, an imprfect but ImageJ free stitching of the tiles can be performed using the below

This is not recommended as there will be overlap related pixel duplications

If you still want to do this for some reason, like a quick look at your tissue, the maxBind option will max project the overlapping areas which looks nice but takes a long time and will definately duplicate pixels. If it is turned to False, the script will run much faster by simply placing image tiles in the predicted locations, this will look worse and will in some case duplicate pixels and in others delete them.